# Table of Contents
- The weather-nominal dataset
- 1. Generating frequent patterns
    - The apriori algorithm
    - The FPgrowth algorithm
- 2. Association rules generation and evaluation


In [ ]:
import os
import pandas as pd

# The weather-nominal dataset

Load the weather-nominal dataset: it is an extremely simple dataset with 13 entries and 5 attributes.

In [ ]:
data = pd.read_csv('dataset/weather-nominal.csv')
data

Association Rule Mining can be considered as a two-step process:
1. **find all frequent itemsets**: impose a predefined *minimum support* (min sup.).
2. **generate *strong* association rules from the freqent itemsets**: typically, association rules are considered interesting if they satisfy both a *minimum support* threshold and a *minimum confidence* threshold.

# 1. Generating frequent patterns


Frequent pattern analysis is beyond the scope of the `scikit-learn` library. In this notebook we will resort to `mlxtend` ([machine learning extension](http://rasbt.github.io/mlxtend/)), one of the third party libraries that implement the most popular frequent pattern mining algorithms.

In [ ]:
from mlxtend.frequent_patterns import apriori

In [ ]:
for column in data.columns:
    data[column]=data[column].apply(lambda x: f'{column}_{x}')

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.describe()

### The apriori algorithm

Apriori is a popular algorithm for extracting frequent itemsets with applications in association rule learning. The apriori algorithm has been designed to operate on databases containing transactions, such as purchases by customers of a store. 

An itemset is considered as "frequent" if it meets a user-specified support threshold. For instance, if the support threshold is set to 0.5 (50%), a frequent itemset is defined as a set of items that occur together in at least 50% of all transactions in the database.



**Encoded format**

The allowed values for a DataFrame provided as input at mlxtend's frequent pattern mining algorithms are either 0/1 or True/False (i.e., boolean vector). 

This encoding complies with the scenario of *market basket analysis*, in which we think of the universe as the set of items available at the store, and each item as a Boolean variable representing the presence or absence of that item. Each basket can be represented by a Boolean vector of values assigned to these variables.

The TransactionEncoder converts item lists into transaction data for frequent itemset mining (simply transforms the input dataset into a one-hot encoded NumPy boolean array)

In [ ]:
from mlxtend.preprocessing import TransactionEncoder 
te = TransactionEncoder()
te_data = te.fit(data.values).transform(data.values)
df = pd.DataFrame(te_data, columns=te.columns_)
df

Notice that the same encoding can be obtained with `sklearn.preprocessing.OneHotEncoder` and `pandas.get_dummies()`.

Now, obtain the items and itemsets with at least MinSup support (e.g., MinSup = 0.2):



In [ ]:
from mlxtend.frequent_patterns import apriori



In [ ]:
apriori?
# - requires setting of min_support
# - requires one-hot encoded dataframe

In [ ]:
freq_itemset = apriori(df, min_support=0.2, use_colnames=True,verbose = True)

In [ ]:
freq_itemset

In [ ]:
sorted(freq_itemset.support.unique())

In [ ]:
[(f'{x}/14={x/14}') for x in range(3,10)]

The type of the itemset value is `frozenset`.

In [ ]:
freq_itemset.itemsets.values[0]

Differently from the classical python `set`, the `frozenset` type is immutable and hashable — its contents cannot be altered after it is created; it can therefore be used as a dictionary key or as an element of another set.

We can leverage the power of pandas to conveniently analyse/filter the results. For instance, we can create the DataFrame of frequent itemsets via apriori and then add a new column that stores the length of each itemset:

In [ ]:
freq_itemset['length'] = freq_itemset['itemsets'].apply(lambda x: len(x))
freq_itemset

Filter the results based on some desired criteria (e.g., selects only the *k*-itemset with *k*>2)

In [ ]:
freq_itemset[(freq_itemset['length']>2)&(freq_itemset['support']>= 0.25)]

In [ ]:
freq_itemset[freq_itemset['itemsets'].apply(lambda x: 'play_yes' in x)]

Note: as we are dealing with frozensets, the order does not matter.

In [ ]:
freq_itemset[freq_itemset['itemsets']=={"play_yes", "outlook_overcast"}]

In [ ]:
freq_itemset[freq_itemset['itemsets']=={"outlook_overcast","play_yes"}]


### The FPGrowth algorithm

FP-Growth is another algorithm for extracting frequent itemsets with applications in association rule learning. It emerged as an efficient alternative to the Apriori algorighm.

In [ ]:
from mlxtend.frequent_patterns import fpgrowth

In [ ]:
fpgrowth?
# - again, requires setting of only one parameter: min_support
# - requires one-hot encoded dataframe

In [ ]:
fpgrowth(df, min_support=0.2,use_colnames=True,verbose = True)

Note that, given a fixed minsup, the set of frequent patterns mined by *Apriori* and *FP-Growth* is **exactly the same**

In [ ]:
%timeit -n 100 -r 10 apriori(df, min_support=0.1)

In [ ]:
%timeit -n 100 -r 10 fpgrowth(df, min_support=0.1)

The dataset is extremely small and the difference is limited.
Anyway, fpgrowth takes typically less time, since it requires just two scans of the database.

## 2. Association rules generation and evaluation

An association rule is an implication expression of the form $X \rightarrow Y$, where $X$ and $Y$ are disjoint itemsets.

Association rules can be generated as follows:
- for each frequent itemset $l$, generate all nonempty subset of $l$
- for every nonempty subset $s$ of $l$, output the rule "$s \rightarrow (l-s)$" if $\frac{\text{support}(l)}{\text{support}(s)}>\text{min_conf}$

As the rules are generated from frequent itemsets, each one automatically satisfy the *minimum support*.

In the following we generate association rules from the frequent itemsets.

In [ ]:
from mlxtend.frequent_patterns import association_rules

In [ ]:
association_rules?

In [ ]:
association_rules(freq_itemset, metric="confidence", min_threshold=0.7)

Let $A \rightarrow C$ be a rule ($A$ and $C$ stands for the antecedent and the consequents, respectively).

The table produced by the association rule mining algorithm contains three different support metrics:
- **antecedent support**: proportion of transactions that contain the antecedent $A$  

$\quad \text{support}(A) \quad \text{range:}\; [0,1]$
- **consequent support**:  proportion of transactions that contain the consequent $C$ 

$\quad \text{support}(C) \quad \text{range:}\; [0,1]$
- **support**: computes the support of the combined itemset $A \cup C$  

$\quad \text{support}(A\rightarrow C)=\text{support}(A \cup C) \quad \text{range:}\; [0,1]$



Many association rule mining algorithms employ the *support-confidence* framework. Indeed, we find also confidence among the metrics:

- **confidence**: probability of seeing the consequent in a transaction given that it also contains the antecedent. 

$\quad \text{confidence}(A \rightarrow C)  = \frac{\text{support}(A \rightarrow C)}{\text{support}(A)} \quad \text{range:}\; [0,1]$

The *support-confidence* framework, however, may still lead to rules that are uninteresting to the users.
Different metrics have been developed to supplement such a framework for the evaluation of an association rule. The current implementation of `mlxtend` makes use of confidence, lift, leverage and conviction metrics, thus enabling a *support-confidence-correlation* framework.


- **lift**: how many times more often $A$ and $C$ occur together than expected if they were statistically independent 

$\quad \text{lift}(A \rightarrow C) = \text{lift}(C \rightarrow A) = \frac{\text{confidence}(A \rightarrow C)}{\text{support}(C)} = \frac{\text{confidence}(C \rightarrow A)}{\text{support}(A)}\quad \text{range:}\; [0,\infty]$

- **leverage**: difference between the observed frequency of A and C appearing together and the frequency that would be expected if A and C were independent. A leverage value of 0 indicates independence.

$\quad \text{leverage}(A \rightarrow C) = \text{support}(A \rightarrow C) - \text{support}(A) \times \text{support}(C) \quad \text{range:}\; [-1,1]$

- **conviction**: high conviction value means that the consequent is highly depending on the antecedent. Similar to lift, if items are independent, the conviction is 1.

$\quad \text{conviction}(A \rightarrow C) = \frac{1-\text{support}(C)}{1-\text{confidence}(A \rightarrow C)}  \quad \text{range:}\; [0,\infty]$

In [ ]:
association_rules(freq_itemset, metric="lift", min_threshold=1.2)

In [ ]:
rules = association_rules(freq_itemset, metric="lift", min_threshold=1.1)
rules

In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

In [ ]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]